In [1]:
import subprocess
import shlex
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
jar_path = '../SentiStrength.jar'
senti_data_path = '../SentStrength_Data/'

In [3]:
df = pd.read_csv('../../dados/instagram/instagram_comments.csv')

In [4]:
def RateSentiment(comments):
    p = subprocess.Popen(shlex.split("java -jar '" + jar_path + "' stdin sentidata '" + senti_data_path + "' trinary"),
                        stdin=subprocess.PIPE,
                        stdout=subprocess.PIPE,
                        stderr=subprocess.PIPE)
    
    #Troca todos os espaços, \n, \r, etc, por '+'
    comments = ['+'.join(c) for c in comments.str.split()]
    conc_comments = '\n'.join(comments)
    
    #codifica para mandar
    b = bytes(conc_comments, 'utf-8')
    stdout_byte, stderr_text = p.communicate(b)
    
    #decodifica
    stdout_text = stdout_byte.decode("utf-8")
    
    #troca tab por espaço entre as scores e remove outros espaços
    stdout_text = stdout_text.rstrip().replace("\t"," ").replace("\r","")
    
    return stdout_text.split('\n')

In [5]:
sents_str = RateSentiment(df['texto_do_comentario'])

In [6]:
sents_list = [sent.split() for sent in sents_str]
sents_list = [s for s in zip(*sents_list)]

df['sent_pos'] = sents_list[0]
df['sent_neu'] = sents_list[2]
df['sent_neg'] = sents_list[1]

df = df.astype({'sent_pos': int, 'sent_neu': int, 'sent_neg': int})

In [7]:
df.head()

,dono_do_post,id_dono_do_post,texto_do_post,shortcode_do_post,id_comment,data_de_postagem,usuario,id_do_usuario,numero_de_likes,texto_do_comentario,em_resposta_a,respostas,sent_pos,sent_neu,sent_neg
0,dr.galvao,5705666877,💕 Amigos são flores que a vida planta na gente...,B-9r1eQJlOy,17959630714309788,2020-04-14 14:28:50,noelikich,27988898452,0,Grande doutor,NaN,NaN,3,1,-1
1,dr.galvao,5705666877,💕 Amigos são flores que a vida planta na gente...,B-9r1eQJlOy,17879780983563775,2020-04-14 15:09:06,lauzinhamdc,962897611,0,👏👏👏,NaN,NaN,1,0,-1
2,dr.galvao,5705666877,💕 Amigos são flores que a vida planta na gente...,B-9r1eQJlOy,17870336905687675,2020-04-14 16:22:15,conchettaleal,4473843923,0,😍😍😍,NaN,NaN,1,0,-1
3,dr.galvao,5705666877,💕 Amigos são flores que a vida planta na gente...,B-9r1eQJlOy,17901676756454382,2020-04-14 17:48:15,priscila_meronho,5622488991,0,The best 😘,NaN,NaN,1,0,-1
4,dr.galvao,5705666877,💕 Amigos são flores que a vida planta na gente...,B-9r1eQJlOy,18138258631046529,2020-04-15 01:35:31,edson.s.soares,7874269864,0,Tenho muito orgulho de ser seu amigo meu amigo...,NaN,NaN,4,1,-1


In [8]:
df.groupby('dono_do_post').mean()[['sent_pos','sent_neu','sent_neg']]

,sent_pos,sent_neu,sent_neg
dono_do_post,,,
dr.galvao,1.368771,0.129568,-1.132890
dralaindutra,1.457904,0.081284,-1.315878
drfelipeades,1.531707,-0.041003,-1.616866
drfernandoneuro,1.430131,0.220381,-1.104062
drlairribeiro,1.360055,0.059363,-1.256580
imedlkep,1.470954,-0.139004,-1.616183
juliommais,1.617371,0.314554,-1.176056
sitedrauziovarella,1.549128,-0.046366,-1.551121
